# March Madness Predictions

Jason Lott, Brendan LaPuma, Eamon Weingold

In [1]:
import pandas as pd, numpy as np
import tensorflow as tf

2023-03-15 04:42:00.246419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 04:42:00.357074: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-15 04:42:00.361532: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-15 04:42:00.361548: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

Import kenpom stats for every team from years 2002-2017

In [3]:
kenpom_df = pd.read_csv("data/kenpom_2017.csv")

Import tourney game results for every tourney game from years 2002-2017

In [4]:
tgr_df = pd.read_csv("data/MNCAATourneyCompactResults.csv")
tgr_df = tgr_df.loc[tgr_df["Season"] >= 2002]
tgr_df = tgr_df.loc[tgr_df["Season"] <= 2017]

Import kaggle IDs for each team

In [5]:
IDs_df = pd.read_csv("data/MTeams.csv")
IDs_df = IDs_df.iloc[:, np.r_[0,1]]

Change team names in kenpom and tgr to thier kaggle IDs

In [6]:
namechange_dict = {'Cal St.': 'CS',
'Albany': 'SUNY Albany',
'Abilene Christian': 'Abilene Chr',
'American': 'American Univ',
'Arkansas Little Rock': 'Ark Little Rock',
'Arkansas Pine Bluff': 'Ark Pine Bluff',
'Boston University': 'Boston Univ',
'Central Michigan': 'C Michigan',
'Central Connecticut': 'Central Conn',
'Coastal Carolina': 'Coastal Car',
'Eastern Washington': 'E Washington',
'East Tennessee St': 'ETSU',
'Fairleigh Dickinson': 'F Dickinson',
'Florida Atlantic': 'FL Atlantic',
'Florida Gulf Coast': 'FL Gulf Coast',
'George Washington': 'G Washington',
'Illinois Chicago': 'IL Chicago',
'Kent St': 'Kent',
'Monmouth': 'Monmouth NJ',
'Mississippi Valley St': 'MS Valley St',
'Northern Colorado': 'N Colorado',
'North Dakota St': 'N Dakota St',
'Northern Kentucky': 'N Kentucky',
'North Carolina A&T': 'NC A&T',
'North Carolina Central': 'NC Central',
'North Carolina St': 'NC State',
'Northwestern St': 'Northwestern LA',
'Prairie View A&M': 'Prairie View',
'South Carolina St': 'S Carolina St',
'South Dakota St': 'S Dakota St',
'Southern Illinois': 'S Illinois',
'Southeastern Louisiana': 'SE Louisiana',
'Stephen F Austin': 'SF Austin',
'Southern': 'Southern Univ',
'Southern Univ Miss': 'Southern Miss',
'Saint Louis': 'St Louis',
'Texas A&M Corpus Chris': 'TAM C. Christi',
'Troy St': 'Troy',
'Texas Southern Univ': 'TX Southern',
'Louisiana Lafayette': 'Louisiana',
'UTSA': 'UT San Antonio',
'Western Michigan': 'W Michigan',
'Green Bay': 'WI Green Bay',
'Milwaukee': 'WI Milwaukee',
'Western Kentucky': 'WKU',
'College of Charleston': 'Col Charleston',
'Loyola Chicago': 'Loyola-Chicago',
'The Citadel':'Citadel',
'UMKC':"Missouri KC",
'Northern Illinois':'N Illinois',
"Louisiana Monroe":'ULM',
'Middle Tennessee':'MTSU',
'Middle Tennessee St':'MTSU',
'Western Carolina':'W Carolina',
'Eastern Illinois': 'E Illinois',
'FIU':'Florida Intl',
'Tennessee Martin':'TN Martin',
'Western Illinois': 'W Illinois',
'Stephan F Austin':"SF Austin",
'Loyola Marymount':'Loy Marymount',
'Charleston Southern':'Charleston So',
'Bethune Cookman':'Bethune-Cookman',
'Eastern Kentucky':'E Kentucky',
'Maryland Eastern Shore':'MD E Shore',
'Birmingham Southern':'Birmingham So',
'Eastern Michigan':'E Michigan',
'Texas Southern':'TX Southern',
'Georgia Southern': 'Ga Southern',
'Cal St Northridge':'CS Northridge',
'Saint Mary\'s': 'St Mary\'s CA',
'Cal St Fullerton':'CS Fullerton',
'Saint Joseph\'s':'St Joseph\'s PA',
'Southwest Texas St':'Texas St',
'Southeast Missouri St':'SE Missouri St',
'Sacramento St':'CS Sacramento',
'Southwest Missouri St':'Missouri St',
'Texas Pan American':'UTRGV',
'UT Rio Grande Valley':'UTRGV',
'IPFW':'PFW',
'Fort Wayne':'PFW',
'Saint Peter\'s':'St Peter\'s',
'Grambling St':'Grambling',
'Utah Valley St':'Utah Valley',
'Mount St Mary\'s':'Mt St Mary\'s',
'Kennesaw St':'Kennesaw',
'Central Arkansas':'Cent Arkansas',
'Winston Salem St':'W Salem St',
'Cal St Bakersfield':'CS Bakersfield',
'USC Upstate':'SC Upstate',
'SIU Edwardsville':'SIUE',
'Houston Baptist':'Houston Chr',
'Nebraska Omaha':'NE Omaha',
'UMass Lowell':'MA Lowell',
'Little Rock':'Ark Little Rock'}

In [7]:
def refactor_teamnames(team_column):
    teamnames = pd.Series(team_column)
    teamnames = teamnames.str.strip()
    teamnames = teamnames.str.replace('.', '', regex=False)
    teamnames = teamnames.str.replace('\',', '\'', regex=False)
    teamnames = teamnames.replace(namechange_dict)
    
    return teamnames

In [8]:
kenpom_df['Team'] = refactor_teamnames(kenpom_df['Team'])

kenpom_df['Team'] = kenpom_df['Team'].str.lower()

# load the csv file with team names and IDs into a dataframe
teams_df = pd.read_csv('data/MTeamSpellings.csv')

# merge the two dataframes based on the team name column
merged_df = pd.merge(kenpom_df, teams_df, how='left', left_on='Team', right_on='TeamNameSpelling')

# replace the team name column with the ID column
kenpom_df['Team Name'] = kenpom_df['Team']
kenpom_df['Team'] = merged_df['TeamID']


In [9]:
kenpom_df["Team"] = kenpom_df["Team"].astype(int)
kenpom_df['Power5'] = kenpom_df['Conference'].isin(['ACC', 'B10', 'SEC', 'B12', 'P10', 'P12']).astype(int)
kenpom_df["WPct"] = kenpom_df["Wins"] / (kenpom_df["Wins"] + kenpom_df["Losses"])
kenpom_df = kenpom_df.drop(columns=["Conference", "Wins", "Losses", "AdjustO Rank", "AdjustD Rank","AdjustT Rank","Luck Rank","SOS Pyth Rank","SOS OppO Rank","SOS OppD Rank","NCSOS Pyth Rank"])

In [10]:
i = 0
for year in kenpom_df.groupby("Year"):
    year_data = year[1]
    for col in ["Pyth","AdjustO","AdjustD","AdjustT","Luck","SOS Pyth","SOS OppO","SOS OppD","NCSOS Pyth", "WPct"]:
        year_data[col] = (year_data[col] - year_data[col].min()) / (year_data[col].max() - year_data[col].min())
    if i == 0:
        normal_kenpom_df = year_data.copy()
        i += 1
    else:
        normal_kenpom_df = pd.concat([normal_kenpom_df, year_data],ignore_index=True,sort=False)

In [11]:
tgr_df2 = tgr_df.copy()
tgr_df2["Result"] = np.select([tgr_df["WTeamID"] > tgr_df["LTeamID"], tgr_df["WTeamID"] < tgr_df["LTeamID"]], [1, 0])
idx = (tgr_df2["Result"] == 0)
tgr_df2.loc[idx,["WTeamID", "WScore", "LTeamID", "LScore"]] = tgr_df2.loc[idx,["LTeamID", "LScore", "WTeamID", "WScore"]].values
tgr_df2 = tgr_df2.rename(columns={"WTeamID":"HTeamID", "WScore":"HScore"})
tgr_df2 = tgr_df2.drop(columns=["DayNum", "NumOT", "HScore", "LScore", "WLoc"])

Below is the training data table. Contains every tourney game 2002-2017 w/ corresponding kenpom data from that year, and result of that game

In [12]:
tgr_w_kenpom_df = pd.merge(tgr_df2, normal_kenpom_df, how='left', left_on=['Season','HTeamID'], right_on=['Year','Team'])
tgr_w_kenpom_df = pd.merge(tgr_w_kenpom_df, normal_kenpom_df, how='left', left_on=['Season','LTeamID'], right_on=['Year','Team'])
tgr_w_kenpom_df = tgr_w_kenpom_df.drop(columns=['Year_x','Year_y', 'Team_x', 'Team_y'])
cols = ["Rank","Seed","Pyth","AdjustO","AdjustD","AdjustT","Luck","SOS Pyth","SOS OppO","SOS OppD","NCSOS Pyth", "Power5","WPct"]
tgr_w_kenpom_df

,Season,HTeamID,LTeamID,Result,Rank_x,Seed_x,Pyth_x,AdjustO_x,AdjustD_x,AdjustT_x,...,AdjustD_y,AdjustT_y,Luck_y,SOS Pyth_y,SOS OppO_y,SOS OppD_y,NCSOS Pyth_y,Team Name_y,Power5_y,WPct_y
0,2002,1373,1108,1,174,16.0,0.458219,0.529138,0.428169,0.495968,...,0.619718,0.758065,0.842105,0.125076,0.133690,0.884354,0.574879,alcorn st,0,0.736534
1,2002,1194,1104,0,191,15.0,0.438476,0.501166,0.430986,0.645161,...,0.239437,0.556452,0.768421,0.828667,0.780749,0.136054,0.456664,alabama,1,0.859083
2,2002,1364,1112,0,77,14.0,0.612489,0.701632,0.352113,0.370968,...,0.338028,0.681452,0.778947,1.000000,1.000000,0.034014,1.000000,arizona,1,0.782408
3,2002,1457,1181,0,219,16.0,0.395470,0.447552,0.445070,0.471774,...,0.000000,0.818548,0.407018,0.867620,0.882353,0.183673,0.690253,duke,1,0.992774
4,2002,1428,1231,0,45,12.0,0.692225,0.776224,0.295775,0.318548,...,0.067606,0.459677,0.329825,0.996957,0.978610,0.006803,0.885763,indiana,1,0.747073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,2017,1314,1246,1,3,1.0,0.938612,0.868486,0.168478,0.555046,...,0.138587,0.587156,0.586093,0.875857,0.857143,0.143939,0.553841,kentucky,1,0.878815
1041,2017,1376,1196,1,24,7.0,0.819683,0.575682,0.048913,0.431193,...,0.086957,0.449541,0.430464,0.929267,0.844720,0.015152,0.713568,florida,1,0.774121
1042,2017,1376,1211,0,24,7.0,0.819683,0.575682,0.048913,0.431193,...,0.000000,0.500000,0.632450,0.587514,0.590062,0.431818,0.455851,gonzaga,0,1.000000
1043,2017,1332,1314,0,10,3.0,0.887322,0.806452,0.187500,0.357798,...,0.168478,0.555046,0.688742,0.930350,0.950311,0.143939,0.558507,north carolina,1,0.859372


Below will ignore the tournament seed in the training, add back in when available

In [13]:
train_df = tgr_w_kenpom_df.sample(frac = 0.8, random_state = 0)
test_df = tgr_w_kenpom_df.drop(train_df.index)

# remove below 2 lines to include tourney seed again
train_df = train_df.drop(columns = ["Seed_x", "Seed_y"])
test_df = test_df.drop(columns = ["Seed_x", "Seed_y"])

train_df2 = train_df.drop(columns = ["Rank_x", "Rank_y", "Team Name_x", "Team Name_y"])
test_df2 = test_df.drop(columns = ["Rank_x", "Rank_y", "Team Name_x", "Team Name_y"])

# will have to adjust columns selected when using tourney seed
x_train = train_df2.iloc[:, 4:28]
y_train = train_df2[["Result"]]

x_test = test_df2.iloc[:, 4:28]
y_test = test_df2[["Result"]] 

#x_train

Current best model - DO NOT CHANGE UNLESS FOUND A CONSISTENTLY BETTER ONE

In [14]:
# change input dim when including tourney seed
model = Sequential([
    Dense(64, activation="relu", input_dim=22),
    Dense(32, activation='relu'),
    Dense(1, activation= 'sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))



Epoch 1/30
2023-03-15 04:42:03.480606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-15 04:42:03.480636: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-15 04:42:03.480650: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-76c1b032-5d0a-4d37-bf01-322f3e813bd4): /proc/driver/nvidia/version does not exist
2023-03-15 04:42:03.480854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
27/27 [==============================] - 1s 7ms/step 

experiment model

In [15]:
# change input dim when including tourney seed
experimentModel = Sequential([
    Dense(32, activation="relu", input_dim=22),
    Dense(16, activation='relu'),
    Dense(1, activation= 'sigmoid')
])

experimentModel.compile(loss="hinge", optimizer='adam', metrics=['accuracy'])

experimentModel.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/30
27/27 [==============================] - 1s 7ms/step - loss: 0.9586 - accuracy: 0.5909 - val_loss: 1.0014 - val_accuracy: 0.5885
Epoch 2/30
27/27 [==============================] - 0s 3ms/step - loss: 0.9302 - accuracy: 0.6423 - val_loss: 0.9730 - val_accuracy: 0.6842
Epoch 3/30
27/27 [==============================] - 0s 3ms/step - loss: 0.9028 - accuracy: 0.6699 - val_loss: 0.9348 - val_accuracy: 0.6603
Epoch 4/30
27/27 [==============================] - 0s 3ms/step - loss: 0.8774 - accuracy: 0.6890 - val_loss: 0.9099 - val_accuracy: 0.6651
Epoch 5/30
27/27 [==============================] - 0s 4ms/step - loss: 0.8583 - accuracy: 0.6806 - val_loss: 0.8915 - val_accuracy: 0.6699
Epoch 6/30
27/27 [==============================] - 0s 2ms/step - loss: 0.8407 - accuracy: 0.6878 - val_loss: 0.8793 - val_accuracy: 0.6746
Epoch 7/30
27/27 [==============================] - 0s 2ms/step - loss: 0.8287 - accuracy: 0.7045 - val_loss: 0.8635 - val_accuracy: 0.7033
Epoch 8/30
27/27 [==

Results of testing the model below

In [16]:
y_pred = model.predict(x_test)
y_test_pred = experimentModel.predict(x_test)

sillylittlearray = np.column_stack((test_df2["Season"],test_df2["HTeamID"], test_df["Team Name_x"], test_df2["LTeamID"], test_df["Team Name_y"], test_df["Rank_x"], test_df["Rank_y"], y_pred, y_test_pred, y_test))
pd.DataFrame(sillylittlearray, columns = ["Season","HTeamID", "HTeam Name", "LTeamID", "LTeam Name", "HTeamRank", "LTeamRank", "Predictions", "Test Predictions", "Actual result"])

7/7 [==============================] - 0s 807us/step


,Season,HTeamID,HTeam Name,LTeamID,LTeam Name,HTeamRank,LTeamRank,Predictions,Test Predictions,Actual result
0,2002,1332,oregon,1285,montana,9,210,0.976527,0.999932,1
1,2002,1214,hampton,1163,connecticut,116,18,0.044952,0.000701,0
2,2002,1373,siena,1268,maryland,174,3,0.012573,0.000289,0
3,2002,1301,nc state,1277,michigan st,21,32,0.62413,0.794146,1
4,2002,1403,texas tech,1356,s illinois,31,49,0.610911,0.948477,0
...,...,...,...,...,...,...,...,...,...,...
204,2017,1455,wichita st,1246,kentucky,8,4,0.185743,0.009719,0
205,2017,1276,michigan,1257,louisville,20,9,0.328181,0.186211,1
206,2017,1332,oregon,1276,michigan,10,20,0.686895,0.854999,1
207,2017,1462,xavier,1112,arizona,31,18,0.351412,0.177203,1


In [17]:
temp = [[1101,1102]]
for i in range(1101, 1478):
    for j in range(i+1, 1478):
        temp.append([i,j])
temp.remove([1101,1102])
submission_df = pd.DataFrame(temp, columns=["LTeamID","HTeamID"])
lteams = submission_df.LTeamID.values.tolist()
hteams = submission_df.HTeamID.values.tolist()
IDs = []
for i in range(0,70876):
    IDs.append("2023_" + str(lteams[i]) + "_" + str(hteams[i]))
submission_df["ID"] = IDs

import requests
from bs4 import BeautifulSoup
import re

def record_to_percent(s):
    n1, n2 = re.findall('(\\d+)-(\\d+)', s)[0]
    n1, n2 = float(n1), float(n2)
    return n1 / (n1 + n2)

bs = BeautifulSoup(open('kp2023.html'), 'html.parser')
bs.prettify()
df = pd.read_html(str(bs.find('table')))[0]
kenpom2023_df = df.iloc[:, np.r_[0:6, 7, 9, 11, 13, 15, 17, 19]]
kenpom2023_df = kenpom2023_df.copy()
kenpom2023_df.columns = ['Rankk','Team','Conference','W-L','AdjEM','AdjO','AdjD','AdjT','Luck','SOS AdjEM','SOS OppO','SOS OppD','NCSOS']

kenpom2023_df['Seed'] = kenpom2023_df['Team'].str.extract('( \d{1,2})', expand=True)
kenpom2023_df['Seed'] = kenpom2023_df['Seed'].str.strip()
kenpom2023_df['Seed'] = kenpom2023_df['Seed'].astype(float).astype('Int32')

kenpom2023_df['Team'] = kenpom2023_df['Team'].replace(to_replace=r' \d{1,2}', value='', regex=True)

kenpom2023_df

,Rankk,Team,Conference,W-L,AdjEM,AdjO,AdjD,AdjT,Luck,SOS AdjEM,SOS OppO,SOS OppD,NCSOS,Seed
0,1,Houston,Amer,31-3,28.82,118.4,89.6,63.3,0.025,4.21,107.3,103.1,0.63,1
1,2,UCLA,P12,29-5,27.63,114.9,87.2,66.3,-0.001,9.00,109.7,100.7,0.72,2
2,3,Alabama,SEC,29-5,27.54,116.1,88.6,72.7,0.074,11.66,110.9,99.2,10.56,1
3,4,Connecticut,BE,25-8,25.37,119.4,94.1,66.7,-0.061,8.51,109.1,100.6,-1.39,4
4,5,Tennessee,SEC,23-10,24.71,112.7,88.0,65.5,-0.071,8.77,109.5,100.7,3.16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,359,St. Francis NY,NEC,14-16,-21.03,90.2,111.3,66.1,0.082,-14.20,97.6,111.7,-9.29,<NA>
359,360,IUPUI,Horz,5-27,-21.84,95.9,117.7,67.1,-0.076,-5.31,102.9,108.2,-6.55,<NA>
360,361,Green Bay,Horz,3-29,-26.30,90.6,116.9,65.2,0.029,-2.34,104.6,107.0,2.07,<NA>
361,362,Hartford,ind,5-23,-29.75,89.4,119.2,65.5,-0.039,-9.55,99.9,109.5,-9.44,<NA>


In [18]:
kenpom2023_df

,Rankk,Team,Conference,W-L,AdjEM,AdjO,AdjD,AdjT,Luck,SOS AdjEM,SOS OppO,SOS OppD,NCSOS,Seed
0,1,Houston,Amer,31-3,28.82,118.4,89.6,63.3,0.025,4.21,107.3,103.1,0.63,1
1,2,UCLA,P12,29-5,27.63,114.9,87.2,66.3,-0.001,9.00,109.7,100.7,0.72,2
2,3,Alabama,SEC,29-5,27.54,116.1,88.6,72.7,0.074,11.66,110.9,99.2,10.56,1
3,4,Connecticut,BE,25-8,25.37,119.4,94.1,66.7,-0.061,8.51,109.1,100.6,-1.39,4
4,5,Tennessee,SEC,23-10,24.71,112.7,88.0,65.5,-0.071,8.77,109.5,100.7,3.16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,359,St. Francis NY,NEC,14-16,-21.03,90.2,111.3,66.1,0.082,-14.20,97.6,111.7,-9.29,<NA>
359,360,IUPUI,Horz,5-27,-21.84,95.9,117.7,67.1,-0.076,-5.31,102.9,108.2,-6.55,<NA>
360,361,Green Bay,Horz,3-29,-26.30,90.6,116.9,65.2,0.029,-2.34,104.6,107.0,2.07,<NA>
361,362,Hartford,ind,5-23,-29.75,89.4,119.2,65.5,-0.039,-9.55,99.9,109.5,-9.44,<NA>


In [19]:
# load the existing CSV file into a DataFrame and add new rows
altnames_df = pd.concat(
    [pd.read_csv('data/MTeamSpellings.csv'),
    pd.DataFrame([('st thomas', 1472), ('queens', 1474), ('texas a&m commerce', 1477), ('houston christian', 1223), ('liu', 1254)], columns=['TeamNameSpelling', 'TeamID'])]
)

# needed for shit to work, idk why
kenpom2023_df = kenpom2023_df.copy()

kenpom2023_df['Team'] = refactor_teamnames(kenpom2023_df['Team'])

kenpom2023_df['Team'] = kenpom2023_df['Team'].str.lower()

altnames_df

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101
...,...,...
0,st thomas,1472
1,queens,1474
2,texas a&m commerce,1477
3,houston christian,1223


In [20]:
kenpom2023_df = kenpom2023_df.copy()

# merge the two dataframes based on the team name column
merged_df = pd.merge(kenpom2023_df, altnames_df, how='left', left_on='Team', right_on='TeamNameSpelling')

# replace the team name column with the ID column
kenpom2023_df['Team'] = merged_df['TeamID']

kenpom2023_df['team name'] = merged_df['TeamNameSpelling']
kenpom2023_df

,Rankk,Team,Conference,W-L,AdjEM,AdjO,AdjD,AdjT,Luck,SOS AdjEM,SOS OppO,SOS OppD,NCSOS,Seed,team name
0,1,1222,Amer,31-3,28.82,118.4,89.6,63.3,0.025,4.21,107.3,103.1,0.63,1,houston
1,2,1417,P12,29-5,27.63,114.9,87.2,66.3,-0.001,9.00,109.7,100.7,0.72,2,ucla
2,3,1104,SEC,29-5,27.54,116.1,88.6,72.7,0.074,11.66,110.9,99.2,10.56,1,alabama
3,4,1163,BE,25-8,25.37,119.4,94.1,66.7,-0.061,8.51,109.1,100.6,-1.39,4,connecticut
4,5,1397,SEC,23-10,24.71,112.7,88.0,65.5,-0.071,8.77,109.5,100.7,3.16,4,tennessee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,359,1383,NEC,14-16,-21.03,90.2,111.3,66.1,0.082,-14.20,97.6,111.7,-9.29,<NA>,st francis ny
359,360,1237,Horz,5-27,-21.84,95.9,117.7,67.1,-0.076,-5.31,102.9,108.2,-6.55,<NA>,iupui
360,361,1453,Horz,3-29,-26.30,90.6,116.9,65.2,0.029,-2.34,104.6,107.0,2.07,<NA>,wi green bay
361,362,1216,ind,5-23,-29.75,89.4,119.2,65.5,-0.039,-9.55,99.9,109.5,-9.44,<NA>,hartford


In [21]:
#kenpom2023_df = pd.merge(kenpom2023_df, IDs_df, left_on="Team", right_on="TeamName", how="left")
#kenpom2023_df = kenpom2023_df.dropna()
#kenpom2023_df["Team"] = kenpom2023_df["Team ID"]
kenpom2023_df["Team"] = kenpom2023_df["Team"].astype(int)
kenpom2023_df['Power5'] = kenpom2023_df['Conference'].isin(['ACC', 'B10', 'SEC', 'B12', 'P10', 'P12']).astype(int)
kenpom2023_df['WPct'] = kenpom2023_df['W-L'].apply(lambda x: record_to_percent(x))
kenpom2023_df = kenpom2023_df.drop(columns=["Conference", "W-L"])

cols = ["AdjEM","AdjO","AdjD","AdjT","Luck","SOS AdjEM","SOS OppO","SOS OppD","NCSOS", "WPct"]
i = 0
for col in cols:
    kenpom2023_df[col] = (kenpom2023_df[col] - kenpom2023_df[col].min()) / (kenpom2023_df[col].max() - kenpom2023_df[col].min())
if i == 0:
    normal_kenpom2023_df = kenpom2023_df.copy()
    i += 1
else:
    normal_kenpom2023_df = pd.concat([normal_kenpom2023_df, kenpom2023_df],ignore_index=True,sort=False)


submission_df = pd.merge(submission_df, normal_kenpom2023_df, left_on="HTeamID", right_on="Team", how="left")
submission_df = pd.merge(submission_df, normal_kenpom2023_df, left_on="LTeamID", right_on="Team", how="left")
submission_df.drop(columns=["Team_x", "Team_y"], inplace=True)

submission_df

,LTeamID,HTeamID,ID,Rankk_x,AdjEM_x,AdjO_x,AdjD_x,AdjT_x,Luck_x,SOS AdjEM_x,...,AdjT_y,Luck_y,SOS AdjEM_y,SOS OppO_y,SOS OppD_y,NCSOS_y,Seed_y,team name_y,Power5_y,WPct_y
0,1101,1102,2023_1101_1102,143.0,0.548040,0.528000,0.510386,0.299320,0.284247,0.582923,...,0.707483,0.308219,0.584680,0.571429,0.410959,0.495856,<NA>,abilene chr,0.0,0.415131
1,1101,1103,2023_1101_1103,104.0,0.623927,0.626667,0.483680,0.442177,0.486301,0.437105,...,0.707483,0.308219,0.584680,0.571429,0.410959,0.495856,<NA>,abilene chr,0.0,0.415131
2,1101,1104,2023_1101_1104,3.0,0.978462,0.789333,0.041543,0.945578,0.732877,0.908644,...,0.707483,0.308219,0.584680,0.571429,0.410959,0.495856,<NA>,abilene chr,0.0,0.415131
3,1101,1105,2023_1101_1105,305.0,0.325257,0.272000,0.617211,0.646259,0.441781,0.207309,...,0.707483,0.308219,0.584680,0.571429,0.410959,0.495856,<NA>,abilene chr,0.0,0.415131
4,1101,1106,2023_1101_1106,349.0,0.219586,0.181333,0.700297,0.489796,0.496575,0.321504,...,0.707483,0.308219,0.584680,0.571429,0.410959,0.495856,<NA>,abilene chr,0.0,0.415131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70871,1474,1476,2023_1474_1476,335.0,0.254922,0.218667,0.679525,0.510204,0.726027,0.133169,...,0.761905,0.489726,0.404427,0.496894,0.678082,0.358559,<NA>,queens,0.0,0.552196
70872,1474,1477,2023_1474_1477,303.0,0.328285,0.400000,0.753709,0.448980,0.568493,0.269501,...,0.761905,0.489726,0.404427,0.496894,0.678082,0.358559,<NA>,queens,0.0,0.552196
70873,1475,1476,2023_1475_1476,335.0,0.254922,0.218667,0.679525,0.510204,0.726027,0.133169,...,0.714286,0.414384,0.228039,0.329193,0.835616,0.442883,<NA>,southern indiana,0.0,0.496629
70874,1475,1477,2023_1475_1477,303.0,0.328285,0.400000,0.753709,0.448980,0.568493,0.269501,...,0.714286,0.414384,0.228039,0.329193,0.835616,0.442883,<NA>,southern indiana,0.0,0.496629


In [22]:
IDs = submission_df["ID"]
y_pred = model.predict(submission_df.drop(columns = ["HTeamID", "LTeamID", "Rankk_x", "Rankk_y", "ID", "team name_x", "team name_y", "Seed_x", "Seed_y"]))

sillylittlearray = np.column_stack((submission_df["HTeamID"], submission_df["LTeamID"], submission_df["Rankk_x"], submission_df["Rankk_y"], y_pred))

predictions_df = pd.DataFrame(sillylittlearray, columns = ["HTeamID", "LTeamID", "H kp rank", "L kp rank","Predictions"])

predictions_df.dropna(inplace=True)

# create a dictionary mapping IDs to team names using the first dataframe
id_to_team = dict(zip(altnames_df['TeamID'], altnames_df['TeamNameSpelling']))

# apply the dictionary to the second dataframe
predictions_df['HTeam Name'] = predictions_df['HTeamID'].map(id_to_team) 
predictions_df['LTeam Name'] = predictions_df['LTeamID'].map(id_to_team)

predictions_df

2215/2215 [==============================] - 2s 914us/step


,HTeamID,LTeamID,H kp rank,L kp rank,Predictions,HTeam Name,LTeam Name
0,1102.0,1101.0,143.0,192.0,0.776712,air-force,abilene-christian
1,1103.0,1101.0,104.0,192.0,0.870971,akron,abilene-christian
2,1104.0,1101.0,3.0,192.0,0.992859,alabama,abilene-christian
3,1105.0,1101.0,305.0,192.0,0.246984,alabama-am,abilene-christian
4,1106.0,1101.0,349.0,192.0,0.196328,alabama-state,abilene-christian
...,...,...,...,...,...,...,...
70871,1476.0,1474.0,335.0,204.0,0.212413,stonehill,queens
70872,1477.0,1474.0,303.0,204.0,0.355741,texas a&m commerce,queens
70873,1476.0,1475.0,335.0,270.0,0.484595,stonehill,southern indiana
70874,1477.0,1475.0,303.0,270.0,0.668017,texas a&m commerce,southern indiana


In [23]:
bracketpredictor_df = predictions_df.drop(columns=["HTeamID", "LTeamID", "H kp rank", "L kp rank"])

bracketpredictor_df.to_csv("bracketpredictor.csv", index=False)

In [24]:
MPreds_df = predictions_df.copy()
MPreds_df["Predictions"] = MPreds_df["Predictions"].apply(lambda x: 1 - x)
MPreds_df.rename(columns={"Predictions":"Pred"},inplace=True)
MPreds_df.drop(columns=["HTeamID","LTeamID","H kp rank","L kp rank", "HTeam Name", "LTeam Name"], inplace=True)
MPreds_df["ID"] = IDs
cols = MPreds_df.columns.to_list()
cols = cols[-1:] + cols[:-1]
MPreds_df = MPreds_df[cols]

In [28]:
MPreds_df.to_csv("submission.csv", line_terminator='\n', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76c1b032-5d0a-4d37-bf01-322f3e813bd4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>